# 1. 모델 YOLOV5 다운로드

In [ ]:
#clone YOLOv5 and torch
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

# 2. mask 데이터셋을 설치

https://public.roboflow.com/object-detection/mask-wearing

In [ ]:
%mkdir /content/mask
%cd /content/mask
!curl -L "https://public.roboflow.com/ds/q17S9NnLHc?key=UlRVud9ghQ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# 3. 추론(Inference)

## detect.py- yolov5 모델을 이용한 추론

- detect하지 않은 마스크 화일 확인하기.

In [ ]:
%cd /content/mask 
display.Image(filename = '/content/mask/train/images/-1x-1_jpg.rf.8d697c00929e06e6655c08835cc66c02.jpg', width=600)

-yolov5에서 주어진 이미지(bus)를 yolov5에서 제공되는 yolov5s.pt로 detect 해보자.

In [ ]:
%cd /content/yolov5/
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source /content/mask/train/images/-1x-1_jpg.rf.8d697c00929e06e6655c08835cc66c02.jpg

- 감지(detect)한 이미지 출력

In [ ]:
display.Image(filename = '/content/yolov5/runs/detect/exp/-1x-1_jpg.rf.8d697c00929e06e6655c08835cc66c02.jpg', width=600)

- train, test, valid에 있는 이미지 화일의 갯수를 출력

In [ ]:
from glob import glob

train_img_list=glob('/content/mask/train/images/*.jpg')
test_img_list=glob('/content/mask/test/images/*.jpg')
valid_img_list=glob('/content/mask/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

- 현재 data.yaml화일을 열어보자.

In [ ]:
%cat /content/mask/data.yaml

## data.yaml 을 아래와 같이 직접 수정한다.

## train: /content/mask/train/images/
## val: /content/mask/valid/images/

## nc: 2
## names: ['mask', 'no-mask']

- data.yaml 내용이 잘 수정되었는지 확인한다.

In [ ]:
%cat /content/mask/data.yaml

# 5. 학습하기

## - train.py 사용

In [ ]:
%%time
%cd /content/yolov5
!python train.py --img 640 --batch 32 --epochs 10 --data /content/mask/data.yaml --weights yolov5s.pt --name mask_results --cache



# 6. tensorboard 로 학습이 잘 진행되었는지 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 7. 결과값을 확인

- results.png 화일을 출력

In [ ]:
display.Image(filename = '/content/yolov5/runs/train/mask_results/results.png', width=1000)

## val_batch0_labels.jpg 출력

In [ ]:
display.Image(filename = '/content/yolov5/runs/train/mask_results/val_batch0_labels.jpg', width=1000)

# 8. 검증하기(validation)
 ### val.py 로 사용

- 경로 확인

In [ ]:
%pwd

- 검증하기(학습된 모델 best.pt로 mask 데이터셋의 train/images 화일을 검증)

In [ ]:
%cd /content/yolov5/
!python val.py --weights /content/yolov5/runs/train/mask_results/weights/best.pt --data /content/mask/data.yaml --img 640 --iou 0.65 --half

- 검증된 사진 확인하기

In [ ]:
display.Image(filename = '/content/yolov5/runs/val/exp2/val_batch0_labels.jpg', width=1000)

# 9. 추론하기(Inference)

- test 이미지 detect 하기

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/mask_results/weights/best.pt --img 640 --conf 0.4 --source /content/mask/test/images

- 확인하기

In [ ]:
display.Image(filename = '/content/yolov5/runs/detect/exp2/1288126-10255706714jpg_jpg.rf.95f7324cbfd48e0386e0660b5e932223.jpg', width=600)

- 동영상 detect하기

- mask_10s.mp4 동영상을 다운로드하여 detect해보자.

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/mask_results/weights/best.pt --img 640 --conf 0.4 --source /content/mask10s.mp4